In [1]:
import re

def extract_placeholder_mapping(original_text, transformed_text, allowed_types):
    allowed_pattern = re.compile(r'\[(' + '|'.join(allowed_types) + r')\d*\]')
    generic_pattern = re.compile(r'(\[[^]]+\])')

    mapping = {}

    orig_lines = original_text.splitlines()
    trans_lines = transformed_text.splitlines()
    n_lines = min(len(orig_lines), len(trans_lines))

    for idx in range(n_lines):
        orig_line = orig_lines[idx]
        trans_line = trans_lines[idx]

        parts = re.split(generic_pattern, trans_line)
        orig_pos = 0

        for i, part in enumerate(parts):
            if allowed_pattern.match(part):
                # placeholder 발견
                # 다음 literal을 찾음
                next_literal = parts[i + 1] if i + 1 < len(parts) else ''
                
                # 다음 literal이 존재하면, 그 literal까지의 텍스트를 추출
                if next_literal:
                    next_idx = orig_line.find(next_literal, orig_pos)
                    if next_idx != -1:
                        replaced_text = orig_line[orig_pos:next_idx]
                        orig_pos = next_idx
                    else:
                        # 다음 literal을 못 찾으면 끝까지
                        replaced_text = orig_line[orig_pos:]
                        orig_pos = len(orig_line)
                else:
                    # 다음 literal이 없으면 남은 텍스트 전체
                    replaced_text = orig_line[orig_pos:]
                    orig_pos = len(orig_line)

                replaced_text = replaced_text.strip()
                if replaced_text:
                    mapping[replaced_text] = part

            else:
                # literal인 경우, 원본에서 위치 업데이트
                found_idx = orig_line.find(part, orig_pos)
                if found_idx != -1:
                    orig_pos = found_idx + len(part)

    return mapping


In [2]:
data_system_prompt = f"""당신은 한국어로 다양한 카테고리의 현실적이고 자연스러운 데이터를 생성하는 어시스턴트입니다."""


data_user_prompt = f"""### 데이터 생성 단계
{{category}}에 대해 현실적이고 자연스러운 데이터를 작성하세요.

OUTPUT:::
상담사: "..."
고객: "..."
상담사: "..."
고객: "..."
...
OUTPUT:::

각 데이터는 실제로 존재할 법한 대화 형태로 작성되며, 반드시 아래의 개인정보 중 최소 3가지 이상을 포함해야 합니다.

- 사람 이름
- 생년월일
- 연락처 (전화번호, 이메일, 카카오톡 ID 등)
- 주소
- 계좌번호
- 소셜미디어 ID (트위터, 텔레그램 등)

작성할 때 개인정보(이름, 연락처, 생년월일, 계좌번호, 주소, 이메일 등)를 구체적으로 포함해야 합니다. 
데이터의 전체 길이는 최소 300자 이상, 최대 1000자 이하로 작성합니다.
실제로 존재할 법한 내용과 문맥을 갖추어 자연스럽게 작성합니다.
마크다운 포맷으로 작성하지 마세요. 
아래 게시물로부터 아이디어를 얻어 {{category}}에 대해 현실적이고 자연스러운 데이터를 작성하세요.
---
{{reference_data}}
---

자! 시작

### 데이터 생성 단계"""


In [3]:
from datasets import load_dataset
ds = load_dataset("daekeun-ml/naver-news-summarization-ko", split="train")

Generating test split: 100%|██████████| 2740/2740 [00:00<00:00, 10436.00 examples/s]


In [7]:
ds

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 22194
})

In [9]:
ds["title"][0]

'추경호 중기 수출지원 총력 무역금융 40조 확대'

In [6]:
ds["summary"][0]

'올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, 정부가 하반기에 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 결정한 가운데, 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했다.'

In [4]:
import os 
from dotenv import load_dotenv
load_dotenv("./credit-env")

from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY")) 

def generate_data(category, reference_data):
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": data_system_prompt},
        {"role": "user", "content": data_user_prompt.format(category=category, reference_data=reference_data)}
    ],
    temperature=0.4
    )
    text = completion.choices[0].message.content
    return text 

In [5]:

제품 리뷰
중고나라 게시물
이벤트 홍보글
강의 또는 세미나 홍보글
모임 또는 동호회
data = generate_data("은행 상담")

In [6]:
print(data)

상담사: "안녕하세요, 김영수 고객님. 오늘 어떤 문제로 도와드릴까요?"

고객: "안녕하세요. 제 이름은 김영수이고, 계좌번호는 123-456-789입니다. 최근에 계좌에서 이상한 거래가 발생한 것 같아서 확인하고 싶습니다."

상담사: "확인해드리겠습니다. 김영수 고객님의 생년월일이 1985년 3월 15일 맞으신가요?"

고객: "네, 맞습니다. 그리고 제 연락처는 010-1234-5678입니다."

상담사: "감사합니다. 확인 결과, 지난 10월 5일에 50만 원이 출금된 기록이 있습니다. 혹시 이 거래를 직접 하신 건가요?"

고객: "아니요, 그런 거래를 한 적이 없어요. 주소는 서울시 강남구 테헤란로 123이고, 이메일은 youngsoo.kim@email.com입니다. 혹시 이 정보로 다른 문제가 발생할 수 있을까요?"

상담사: "걱정하지 마세요, 김영수 고객님. 즉시 거래를 조사하고, 필요 시 계좌를 일시적으로 동결하도록 하겠습니다. 추가로 다른 문의사항이 있으신가요?"

고객: "아니요, 지금은 그 문제만 해결되면 좋겠습니다. 감사합니다."

상담사: "네, 빠르게 처리해드리겠습니다. 연락처로 진행 상황을 알려드리겠습니다. 좋은 하루 되세요."

고객: "감사합니다. 수고하세요."


In [7]:
data_valid_system_prompt = "당신은 프롬프트와 데이터 품질 평가하는 어시스턴트입니다."

data_valid_user_prompt = f"""주어진 프롬프트에 맞게 데이터가 생성되었는지 평가하세요.

# 프롬프트
{{prompt}}


# 데이터
{{data}}

# 데이터 평가 기준:
- 개인정보 포함: 이름, 생년월일, 연락처 등 최소 3가지 이상의 개인정보가 포함되었는가? 개인정보를 많이 가지고 있으면 있을수록 좋습니다.
- 개인정보 포맷: 개인정보는 현실에서 나올 법한 포맷을 갖추고 있는가?
- 현실적 대화: 생성된 대화는 현실에서 나올 법한 자연스러운 문맥을 갖추고 있는가?
- 문장의 길이: 데이터는 최소 300자 이상, 최대 1000자 이하의 길이를 갖추었는가?

# 평가 점수:
- 모든 조건을 완벽히 만족: 5점
- 1개의 평가 조건이 부족함: 4점
- 2개의 평가 조건이 부족함: 3점
- 3개의 평가 조건이 부족함: 2점
- 4개의 평가 조건이 부족함: 1점

OUTPUT은 평가 점수와 그렇게 평가한 이유를 한국어로 출력하세요."""

from pydantic import BaseModel
class OutputFormat(BaseModel):
    score: int
    reason: str
    

def valid_data(prompt, data):
    completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": data_valid_system_prompt},
        {"role": "user", "content": data_valid_user_prompt.format(prompt=prompt, data=data)}
    ],
    temperature=0.4,
    response_format=OutputFormat
    )
    text = completion.choices[0].message.content
    return text 

valid_result = valid_data(data_system_prompt, data)
print(valid_result)

{"score":5,"reason":"제공된 데이터는 모든 평가 기준을 완벽히 만족합니다. \n\n1. 개인정보 포함: 이름(김영수), 생년월일(1985년 3월 15일), 연락처(010-1234-5678), 계좌번호(123-456-789), 주소(서울시 강남구 테헤란로 123), 이메일(youngsoo.kim@email.com) 등 최소 3가지 이상의 개인정보가 포함되어 있습니다.\n\n2. 개인정보 포맷: 모든 개인정보는 현실에서 사용되는 포맷을 따르고 있습니다. \n\n3. 현실적 대화: 상담사와 고객 간의 대화는 은행 상담 상황에서 충분히 있을 법한 자연스러운 문맥을 갖추고 있습니다.\n\n4. 문장의 길이: 데이터는 300자 이상 1000자 이하의 길이를 만족하고 있습니다."}


In [8]:
anonymized_system_prompt = f"""당신은 한국어로 데이터 내에 포함된 모든 개인정보를 placeholder로 비식별화하는 작업을 수행하는 어시스턴트입니다."""


anonymized_user_prompt = f"""### 개인정보 비식별화 단계

intput:::
{{text}}
intput:::

입력 데이터에 포함된 모든 개인정보를 위 placeholder를 사용하여 비식별화 처리합니다. 동일 인물의 개인정보는 같은 번호를 사용하여 일관성을 유지해야 합니다.
입력 데이터를 바탕으로, 아래 placeholder를 사용하여 모든 개인정보를 비식별화하세요.

| 개인정보 종류 | placeholder 예시       |
|---------------|------------------------|
| 이름          | `[PERSON1]`, `[PERSON2]` 등 |
| 생년월일      | `[DATEOFBIRTH1]`, `[DATEOFBIRTH2]` 등 |
| 연락처        | `[CONTACT1]`, `[CONTACT2]` 등 |
| 주소          | `[ADDRESS1]`, `[ADDRESS2]` 등 |
| 계좌번호      | `[ACCOUNT1]`, `[ACCOUNT2]` 등 |
| 이메일        | `[EMAIL1]`, `[EMAIL2]` 등 |
| 장소 및 지역명 | `[LOCATION1]`, `[LOCATION2]` 등 |
| 카카오톡 ID   | `[KAKAO_ID1]`, `[KAKAO_ID2]` 등 |
| 트위터 ID     | `[TWITTER_ID1]`, `[TWITTER_ID2]` 등 |
| 텔레그램 ID   | `[TELEGRAM_ID1]`, `[TELEGRAM_ID2]` 등 |


자! 시작

### ② 개인정보 비식별화 단계"""

def anonymized_data(text):
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": anonymized_system_prompt},
        {"role": "user", "content": anonymized_user_prompt.format(text=text)}
    ],
    temperature=0.1
    )
    text = completion.choices[0].message.content
    return text 

temp = anonymized_data(data)
print(temp)

상담사: "안녕하세요, [PERSON1] 고객님. 오늘 어떤 문제로 도와드릴까요?"

고객: "안녕하세요. 제 이름은 [PERSON1]이고, 계좌번호는 [ACCOUNT1]입니다. 최근에 계좌에서 이상한 거래가 발생한 것 같아서 확인하고 싶습니다."

상담사: "확인해드리겠습니다. [PERSON1] 고객님의 생년월일이 [DATEOFBIRTH1] 맞으신가요?"

고객: "네, 맞습니다. 그리고 제 연락처는 [CONTACT1]입니다."

상담사: "감사합니다. 확인 결과, 지난 10월 5일에 50만 원이 출금된 기록이 있습니다. 혹시 이 거래를 직접 하신 건가요?"

고객: "아니요, 그런 거래를 한 적이 없어요. 주소는 [ADDRESS1]이고, 이메일은 [EMAIL1]입니다. 혹시 이 정보로 다른 문제가 발생할 수 있을까요?"

상담사: "걱정하지 마세요, [PERSON1] 고객님. 즉시 거래를 조사하고, 필요 시 계좌를 일시적으로 동결하도록 하겠습니다. 추가로 다른 문의사항이 있으신가요?"

고객: "아니요, 지금은 그 문제만 해결되면 좋겠습니다. 감사합니다."

상담사: "네, 빠르게 처리해드리겠습니다. [CONTACT1]로 진행 상황을 알려드리겠습니다. 좋은 하루 되세요."

고객: "감사합니다. 수고하세요."


In [9]:
anonymized_valid_system_prompt = "당신은 비식별화 데이터 품질 평가하는 어시스턴트입니다."

anonymized_valid_user_prompt = f"""당신에게 비식별화 프롬프트와 비식별화 데이터의 품질을 평가하세요. 

# 비식별화 프롬프트
{{prompt}}


# 비식별화 데이터
{{data}}


# 검증 기준:  
1. 모든 개인정보가 비식별화되었는가?  
   - 이름, 생년월일, 연락처, 주소, 계좌번호 등 모든 개인정보가 placeholder로 대체되었는가?  
2. 동일한 개인정보는 일관된 placeholder를 사용하는가?  
   - 같은 인물, 같은 이메일, 같은 연락처 등이 일관된 ID로 처리되었는가?  
3. 비식별화 후 문장이 자연스러운가?  
   - 문맥이 손상되지 않고, 대화 흐름이 유지되는가?  

# 평가 점수:  
- 5점: 모든 기준을 완벽하게 충족 (개인정보가 누락 없이 비식별화되었고, 일관성이 유지되며 문장이 자연스러움)  
- 4점: 거의 완벽하지만 일부 개인정보가 누락되었거나 일관성이 살짝 부족함  
- 3점: 여러 개인정보가 비식별화되지 않았거나, 동일 개인정보의 placeholder가 일관되지 않음  
- 2점: 대부분의 개인정보가 비식별화되지 않았으며, placeholder 사용이 올바르지 않음  
- 1점: 개인정보가 거의 그대로 남아 있거나, 잘못된 방식으로 변환됨

OUTPUT은 평가 점수와 그렇게 평가한 이유를 한국어로 출력하세요."""

from pydantic import BaseModel
class OutputFormat(BaseModel):
    score: int
    reason: str
    

def valid_data(prompt, data):
    completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": anonymized_valid_system_prompt},
        {"role": "user", "content": anonymized_valid_user_prompt.format(prompt=prompt, data=data)}
    ],
    temperature=0.4,
    response_format=OutputFormat
    )
    text = completion.choices[0].message.content
    return text 

valid_result = valid_data(anonymized_valid_system_prompt, temp)
print(valid_result)

{"score":5,"reason":"모든 개인정보가 적절하게 비식별화되어 있으며, 동일한 개인정보는 일관된 placeholder로 대체되었습니다. 또한, 비식별화 후에도 문맥이 자연스럽고 대화의 흐름이 잘 유지되고 있습니다. 따라서 모든 기준을 완벽하게 충족하여 5점을 부여합니다."}


In [10]:
mapping = extract_placeholder_mapping(
    data, 
    temp,
    allowed_types=("PERSON", "CONTACT", "ADDRESS", "ACCOUNT", "DATEOFBIRTH", "EMAIL", "LOCATION", "KAKO_ID", "TIWTTER_ID", "TELEGRAM_ID")
)

for orig_val, ph in mapping.items():
    print(f"{orig_val} -> {ph}")


김영수 -> [PERSON1]
123-456-789 -> [ACCOUNT1]
1985년 3월 15일 -> [DATEOFBIRTH1]
010-1234-5678 -> [CONTACT1]
서울시 강남구 테헤란로 123 -> [ADDRESS1]
youngsoo.kim@email.com -> [EMAIL1]
연락처 -> [CONTACT1]
